In [2]:
import os
import shutil

In [1]:
file_path = '/home/max/stayahead/mutations/sequence_datasets/1/comp_performance.fasta'

In [4]:
directory_path = '/home/max/stayahead/out_tmp/alphafold/batch1'
output_file_path = '/home/max/stayahead/out_tmp/alphafold/candidates.fasta'

fasta_files = [f for f in os.listdir(directory_path) if f.endswith('.fasta')]

with open(output_file_path, 'w') as outfile:
    # Iterate over each file and append its contents to the output file
    for file_name in fasta_files:
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'r') as infile:
            sequence = ''
            for line in infile:
                if line.startswith('>'):
                    # If there is an existing sequence, write it before starting a new header
                    if sequence:
                        outfile.write(sequence + '\n')
                    # Write the header
                    outfile.write(line)
                    sequence = ''
                else:
                    # Remove newline and concatenate sequence parts
                    sequence += line.strip()
            # Don't forget to write the last sequence if the file ends
            if sequence:
                outfile.write(sequence + '\n')

print("Merging complete. All fasta files have been combined into 'baits.fasta'.")

Merging complete. All fasta files have been combined into 'baits.fasta'.


In [11]:
with open(file_path, 'r') as f:
    lines = f.readlines()
    for i in range(140, 280, 2):
        description = lines[i].strip()
        name = description[1:]
        with open (f'/home/max/stayahead/mutations/sequence_datasets/1/sep_files/{name}.fasta', 'w') as out:
            out.write(description + '\n')
            out.write(lines[i+1])

In [2]:
def split_fasta(input_file):
    with open(input_file, 'r') as file:
        sequence = ''
        header = ''
        for line in file:
            if line.startswith('>'):
                if sequence:
                    # Write the previous sequence to a file
                    with open(f'{header}.fasta', 'w') as output_file:
                        output_file.write(f'>{header}\n{sequence}')
                    sequence = ''
                header = line[1:].strip()  # Remove '>' and newline character
            else:
                sequence += line.strip()  # Add sequence lines
        # Write the last sequence to a file
        if sequence:
            with open(f'{header}.fasta', 'w') as output_file:
                output_file.write(f'>{header}\n{sequence}')

# Call the function with the path to your FASTA file
split_fasta('/home/max/stayahead/mutations/sequence_datasets/3/ACE2_1step.fasta')

In [2]:
def split_fasta(input_file, output_dir):
    # Extract the base name of the file without the .fasta extension
    base_name = os.path.basename(input_file)
    if base_name.endswith('.fasta'):
        base_name = base_name[:-6]
    
    # Create a new directory based on the base name inside the specified output directory
    target_dir = os.path.join(output_dir, base_name)
    os.makedirs(target_dir, exist_ok=True)
    
    with open(input_file, 'r') as file:
        sequence = ''
        header = ''
        for line in file:
            if line.startswith('>'):
                if sequence:
                    # Write the previous sequence to a file in the new directory
                    with open(os.path.join(target_dir, f'{header}.fasta'), 'w') as output_file:
                        output_file.write(f'>{header}\n{sequence}')
                    sequence = ''
                header = line[1:].strip()  # Remove '>' and newline character
            else:
                sequence += line.strip()  # Add sequence lines
        # Write the last sequence to a file in the new directory
        if sequence:
            with open(os.path.join(target_dir, f'{header}.fasta'), 'w') as output_file:
                output_file.write(f'>{header}\n{sequence}')


In [5]:
def process_directory(input_dir, output_dir):
    # Loop through all files in the input directory
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.fasta'):
            file_path = os.path.join(input_dir, file_name)
            split_fasta(file_path, output_dir)

In [6]:
process_directory('/home/max/stayahead/mutations/sequence_datasets/3/alt_variants', '/home/max/stayahead/mutations/sequence_datasets/3/sep_files')

In [2]:
def move_fasta_files(input_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Walk through all directories and subdirectories in the input directory
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.fasta'):
                # Construct the full file path
                file_path = os.path.join(root, file)
                # Move the file to the output directory
                shutil.move(file_path, os.path.join(output_dir, file))


In [3]:
move_fasta_files('/home/max/stayahead/mutations/sequence_datasets/3/alt_variants/sep_files', '/home/max/stayahead/mutations/sequence_datasets/3/alt_variants/sep_alt_variants')

In [2]:
import pandas as pd
import os

def merge_csv_files(directory, output_file):
    # List to hold data from each file
    data_frames = []

    # Iterate through each file in the directory
    for file in os.listdir(directory):
        # Check if the file is a CSV
        if file.endswith('.csv'):
            file_path = os.path.join(directory, file)
            # Read the CSV file and append to the list
            df = pd.read_csv(file_path)
            data_frames.append(df)

    # Concatenate all data frames into a single one
    merged_df = pd.concat(data_frames, ignore_index=True)

    # Save the merged data frame to a new CSV file
    merged_df.to_csv(output_file, index=False)

# Example usage
directory = '/home/max/stayahead/snellius2/outputs/analysis/alphafold/all_batches'  # Update this path to the directory containing your CSV files
output_file = '/home/max/stayahead/snellius2/outputs/analysis/alphafold/all_batches/results_1step_AF_15_05.csv'
merge_csv_files(directory, output_file)

In [5]:
def remove_prefix_from_seq_id(input_filename, output_filename):
    # Read the CSV file
    df = pd.read_csv(input_filename)
    
    # Remove 'ACE2-' prefix from each entry in the 'seq_id' column
    df['seq_id'] = df['seq_id'].str.replace('ACE2-', '')
    
    # Write the modified DataFrame back to a new CSV file
    df.to_csv(output_filename, index=False)
    print(f"Updated file created as {output_filename}")

input_file = '/home/max/stayahead/snellius2/outputs/apbs/esm/epc_1step_esm.csv'
output_file = '/home/max/stayahead/snellius2/outputs/apbs/esm/epc_1step_esm_clean.csv'
remove_prefix_from_seq_id(input_file, output_file)

Updated file created as /home/max/stayahead/snellius2/outputs/apbs/esm/epc_1step_esm_clean.csv


In [12]:
def merge_csv_on_seq_id(file1, file2, output_file):
    # Load the two CSV files into pandas DataFrames
    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)
    
    # Merge the DataFrames on the 'seq_id' column
    merged_df = pd.merge(df1, df2, on='seq_id', how='left')
    
    # Save the merged DataFrame to a new CSV file
    merged_df.to_csv(output_file, index=False)
    print(f"Merged file saved as {output_file}")

file1 = '/home/max/stayahead/snellius2/outputs/analysis/dataset/tmp/merged_1step_esm.csv'
file2 = '/home/max/stayahead/evolution/data/escape_all_formatted.csv'
output_file = '/home/max/stayahead/snellius2/outputs/analysis/dataset/1step_esm_ablink.csv'
merge_csv_on_seq_id(file1, file2, output_file)

Merged file saved as /home/max/stayahead/snellius2/outputs/analysis/dataset/1step_esm_ablink.csv


In [4]:
import csv
import pandas as pd

df = pd.read_csv('/home/max/stayahead/analysis/AF_1step_pred_wuhan_v1_15-06.csv')
ids = df['seq_id'].tolist()

In [7]:
print(ids)

['P5N', 'V75A', 'T146A', 'R25F', 'Y141K', 'Q166D', 'F154G', 'E74P', 'V179Q', 'N155A', 'G49Y', 'S67W', 'L186P', 'F183H', 'D66M', 'S51Q', 'V171N', 'C47A', 'C59M', 'E139G', 'K130G', 'G84H', 'S145A', 'S182W', 'V171R', 'G153L', 'V30L', 'S43F', 'V9F', 'P195R', 'V179F', 'P94D', 'A31M', 'F60V', 'K24S', 'G170F', 'Y64G', 'C47N', 'F183S', 'R25Y', 'S162H', 'S67D', 'L181K', 'G170E', 'T44I', 'K85N', 'S106H', 'C4S', 'N2F', 'G99T', 'S34G', 'A143F', 'V178N', 'R14A', 'I70G', 'R177C', 'A79I', 'S145V', 'Q166S', 'N28M', 'F158V', 'C193F', 'S41P', 'L120E', 'A87I', 'G7A', 'Y91T', 'G153D', 'C4V', 'K112C', 'K24M', 'G99L', 'Y163T', 'L55I', 'C100F', 'W21Q', 'E133P', 'N38L', 'G72D', 'L109W', 'R23T', 'G81Q', 'P94L', 'L93D', 'L123Q', 'D32Y', 'N169H', 'P52L', 'Q174L', 'G194Y', 'Q166R', 'I86S', 'R25Q', 'Y157E', 'A40D', 'N169L', 'T1L', 'L93I', 'G7T', 'T98A', 'Y173L', 'N155K', 'N2L', 'L160A', 'F60C', 'L185M', 'N90R', 'T44Q', 'P52M', 'V113P', 'K85R', 'Q77I', 'R122P', 'G7Y', 'V9T', 'I26Q', 'Y37C', 'N169K', 'R14N', 'N38S',

In [12]:
import os

# Paths to the directories
directory_1 = '/home/max/stayahead/out_tmp/pd_output'
directory_2 = '/home/max/stayahead/out_tmp/alphafold/1step_all'

# Get the set of directory names in directory_1
dir1_names = {os.path.splitext(f)[0] for f in os.listdir(directory_1)}
dir1_seq_ids = {f.split('_')[-1] for f in dir1_names}
dir1_ids = set(ids)

# dir2_names = {name for name in os.listdir(directory_2) if os.path.isdir(os.path.join(directory_2, name))}
dir2_names = {f.split('.')[0] for f in os.listdir(directory_2)}
dir2_ids = {f.split('-')[-1] for f in dir2_names}
print(dir2_ids)

# Find the missing directories based on partial matches
missing_dirs = {name for name in dir2_names if not any(d1_name.startswith(name) for d1_name in dir1_seq_ids)}
missing_dirs = {name for name in dir2_ids if not any(d1_name.startswith(name) for d1_name in dir1_ids)}

# Print the missing directories
print("Missing directories:")
print(len(missing_dirs))
print(missing_dirs)

{'C156T', 'N118H', 'G49R', 'S39E', 'A20N', 'S43C', 'R71Y', 'L181V', 'N107D', 'L120E', 'R177I', 'L109N', 'R122S', 'V151F', 'Q174M', 'V35M', 'F68M', 'N22Q', 'G144L', 'T44H', 'A143Q', 'V192D', 'V151G', 'V101F', 'V75R', 'V171N', 'P195I', 'C193F', 'E8I', 'S43Y', 'V151W', 'G150Q', 'T53W', 'R14C', 'V63I', 'C193D', 'F60H', 'L181S', 'G194W', 'N149D', 'G72A', 'G172T', 'N116I', 'L123R', 'S106E', 'S137H', 'C59E', 'Y64S', 'D57K', 'T61H', 'F60A', 'F154C', 'Y173A', 'G115Y', 'N118K', 'C156N', 'P175I', 'L181W', 'V63W', 'N108A', 'V192F', 'E133W', 'T98W', 'K130F', 'S67D', 'A40G', 'R23D', 'S41V', 'T13G', 'C193E', 'V179C', 'V178W', 'F97Y', 'R122K', 'G194A', 'A20M', 'P175Y', 'N28E', 'S111F', 'L186V', 'L58Y', 'N2V', 'Y121S', 'F124H', 'C156K', 'V50C', 'I140R', 'D32C', 'F183W', 'G170Y', 'P189Y', 'G81N', 'L58V', 'R125A', 'V69D', 'D66M', 'G49D', 'V50M', 'V192G', 'P189N', 'D135P', 'K130Q', 'D95W', 'V35L', 'F132E', 'S43D', 'N105V', 'A65H', 'A143P', 'D95I', 'Y37G', 'Q161V', 'T53S', 'F10K', 'S41T', 'S67W', 'F124C', 

In [37]:
import os

directory_1 = '/home/max/stayahead/out_tmp/alphafold/pd/1step100_fastas'
directory_2 = '/home/max/stayahead/out_tmp/alphafold/feat100_2'
directory_3 = '/home/max/stayahead/out_tmp/alphafold/feat100_1'

# Get the set of directory names in directory_1 (excluding .fasta files)
dir1_names = {os.path.splitext(f)[0] for f in os.listdir(directory_1) if f.endswith('.fasta')}
print(f"Directory 1 names: {dir1_names}")

# Get the set of directory names in directory_2 (only directories)
dir2_names = {name for name in os.listdir(directory_2) if os.path.isdir(os.path.join(directory_2, name))}
dir3_names = {name for name in os.listdir(directory_3) if os.path.isdir(os.path.join(directory_3, name))}
print(f"Directory 2 names: {dir2_names}")

combined_names = dir2_names.union(dir3_names)
# Find the missing directories based on partial matches
# missing_dirs = {name for name in dir2_names if not any(name in d1_name for d1_name in dir1_names)}
missing_dirs = {name for name in dir1_names if not any(com_name in name for com_name in combined_names)}

# Print the missing directories
print("Missing directories:")
print(len(missing_dirs))
print(missing_dirs)

Directory 1 names: {'ACE2-N90S', 'ACE2-N149Y', 'ACE2-G81V', 'ACE2-N105W', 'ACE2-G115D', 'ACE2-T138N', 'ACE2-N105C', 'ACE2-N118S', 'ACE2-A103Q', 'ACE2-P189H', 'ACE2-C156H', 'ACE2-E139I', 'ACE2-P80Q', 'ACE2-W104R', 'ACE2-Q82F', 'ACE2-T168G', 'ACE2-Y141W', 'ACE2-L120W', 'ACE2-G99K', 'ACE2-Y91T', 'ACE2-E152S', 'ACE2-A143M', 'ACE2-L123C', 'ACE2-L185W', 'ACE2-F124Q', 'ACE2-G81A', 'ACE2-L129A', 'ACE2-Q174T', 'ACE2-D135Y', 'ACE2-A188V', 'ACE2-G114I', 'ACE2-S162L', 'ACE2-S162R', 'ACE2-Q174E', 'ACE2-Q166C', 'ACE2-S106A', 'ACE2-E184W', 'ACE2-G150Q', 'ACE2-V151D', 'ACE2-G153N', 'ACE2-P80A', 'ACE2-S162M', 'ACE2-F165K', 'ACE2-D135M', 'ACE2-I140V', 'ACE2-C148V', 'ACE2-K112T', 'ACE2-Y121I', 'ACE2-G99R', 'ACE2-C193S', 'ACE2-K112I', 'ACE2-D96I', 'ACE2-G150A', 'ACE2-G150Y', 'ACE2-A143Y', 'ACE2-L181A', 'ACE2-A190H', 'ACE2-L186E', 'ACE2-P131T', 'ACE2-G144C', 'ACE2-S145E', 'ACE2-T83V', 'ACE2-G114P', 'ACE2-D88F', 'ACE2-Y163M', 'ACE2-K92P', 'ACE2-F183Y', 'ACE2-F154P', 'ACE2-N169Y', 'ACE2-R125D', 'ACE2-C193W',

In [40]:
import shutil
fasta_source_dir = '/home/max/stayahead/out_tmp/alphafold/pd/1step100_fastas'
output_dir = '/home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas'

# Copy the fasta files
for missing_dir in missing_dirs:
    fasta_file = f"{missing_dir}.fasta"
    source_path = os.path.join(fasta_source_dir, fasta_file)
    if os.path.isfile(source_path):
        shutil.copy(source_path, output_dir)
        print(f"Copied {fasta_file} to {output_dir}")
    else:
        print(f"{fasta_file} not found in {fasta_source_dir}")

print("Finished copying fasta files.")

Copied ACE2-C193I.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-G194L.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-V192R.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-A190V.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-C193Y.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-C193H.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-P195M.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-A190W.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-P195D.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-C193M.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas
Copied ACE2-V192M.fasta to /home/max/stayahead/out_tmp/alphafold/pd/1step100_mis

In [ ]:
# # Get the set of FASTA file names without the .fasta extension in directory_2
# fasta_names = {os.path.splitext(f)[0] for f in os.listdir(directory_2) if f.endswith('.fasta')}

# # Find the missing directories
# missing_dirs = fasta_names - dir1_names

# # Print the missing directories
# print("Missing directories:")
# for missing_dir in missing_dirs:
#     print(missing_dir)

# Get the set of directory names in directory_2
# dir2_names = {f for f in os.listdir(directory_2) if os.path.isdir(os.path.join(directory_2, f))}

In [4]:
name_dir = '/home/max/stayahead/out_tmp/alphafold/variants'
dir1_names = set(os.listdir(name_dir))
with open('/home/max/stayahead/out_tmp/alphafold/candidates_variants.txt', 'w') as f:
    for name in dir1_names:
        f.write(name + '\n')


In [1]:
dir_path = '/home/max/stayahead/out_tmp/alphafold/pd/1step100_missing_fastas'
output_file_path = '/home/max/stayahead/out_tmp/alphafold/pd/candidates_100_missing.fasta'
with open(output_file_path, 'w') as outfile:
    with open('/home/max/stayahead/out_tmp/alphafold/pd/candidates_100_missing.txt', 'r') as f:
        f = f.readlines()
        for name in f:
            name = name.strip()
            fasta_name = name + '.fasta'
            with open(f'/home/max/stayahead/out_tmp/alphafold/pd/1step100_fastas/{fasta_name}', 'r') as infile:
                sequence = ''
                for line in infile:
                    if line.startswith('>'):
                        # If there is an existing sequence, write it before starting a new header
                        if sequence:
                            outfile.write(sequence + '\n')
                        # Write the header
                        outfile.write(line)
                        sequence = ''
                    else:
                        # Remove newline and concatenate sequence parts
                        sequence += line.strip()
                # Don't forget to write the last sequence if the file ends
                if sequence:
                    outfile.write(sequence + '\n')

In [3]:
directory_path = '/home/max/stayahead/out_tmp/alphafold/variants/variants'
output_file_path = '/home/max/stayahead/out_tmp/alphafold/candidates_variants.fasta'

fasta_files = [f for f in os.listdir(directory_path) if f.endswith('.fasta')]

with open(output_file_path, 'w') as outfile:
    # Iterate over each file and append its contents to the output file
    for file_name in fasta_files:
        file_path = os.path.join(directory_path, file_name)
        with open(file_path, 'r') as infile:
            sequence = ''
            for line in infile:
                if line.startswith('>'):
                    # If there is an existing sequence, write it before starting a new header
                    if sequence:
                        outfile.write(sequence + '\n')
                    # Write the header
                    outfile.write(line)
                    sequence = ''
                else:
                    # Remove newline and concatenate sequence parts
                    sequence += line.strip()
            # Don't forget to write the last sequence if the file ends
            if sequence:
                outfile.write(sequence + '\n')

In [15]:
dir1 = '/home/max/stayahead/out_tmp/pd_output'
txt_file_path = '/home/max/stayahead/out_tmp/alphafold/pd/candidates_50_2.txt'

# Read the contents of the text file
with open(txt_file_path, 'r') as file:
    lines = file.readlines()

# Strip whitespace and newline characters from each line
lines = [line.strip() for line in lines]

# Get a list of directory names in the specified directory
dir_names = os.listdir(dir1)

# Filter out lines that have a matching directory in the specified directory
filtered_lines = [line for line in lines if f'ACE2-ectodomain_and_{line}' not in dir_names]

# Write the filtered lines back to the text file
with open(txt_file_path, 'w') as file:
    for line in filtered_lines:
        file.write(f"{line}\n")

In [5]:
fp = '/home/max/stayahead/out_tmp/alphafold/pd/candidates_50_1.txt'
output_dir = '/home/max/stayahead/out_tmp/alphafold/pd/split_files/'

# Create the output directory if it doesn't exist
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

with open(fp, 'r') as f:
    lines = f.readlines()

# Calculate the number of files needed
num_files = len(lines) // 25 + (1 if len(lines) % 50 != 0 else 0)

# Write lines to separate files
for i in range(num_files):
    start_line = i * 50
    end_line = start_line + 50
    file_lines = lines[start_line:end_line]
    
    output_file = os.path.join(output_dir, f'candidates_50_1-{i + 1}.txt')
    with open(output_file, 'w') as out_f:
        out_f.writelines(file_lines)

In [ ]:
dir1 = '/home/max/ownCloud - Erik Schultes@universiteitleiden.data.surfsara.nl/LACDR_TKI_LSH (Projectfolder)/alphafold/pulldown/1step/50'

dir_names = os.listdir(dir1)

In [3]:
import os

# Define the paths
fasta_file_path = '/home/max/stayahead/out_tmp/alphafold/candidates_100.fasta'
msa_directory_path = '/home/max/stayahead/out_tmp/alphafold/feat100_1'
filtered_fasta_file_path = '/home/max/stayahead/out_tmp/alphafold/candidates_100-1.fasta'
updated_fasta_file_path = '/home/max/stayahead/out_tmp/alphafold/candidates_100-2.fasta'

# Read the directories in the msa directory
existing_directories = set(os.listdir(msa_directory_path))

# Function to read the fasta file and filter the sequences
def filter_fasta_sequences(fasta_file, existing_dirs):
    filtered_sequences = []
    remaining_sequences = []
    keep_sequence = False
    sequence_name = ""

    with open(fasta_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            if line.startswith(">"):
                sequence_name = line.strip().lstrip(">")
                if sequence_name not in existing_dirs:
                    keep_sequence = True
                    filtered_sequences.append(line)
                else:
                    keep_sequence = False
                    remaining_sequences.append(line)
            else:
                if keep_sequence:
                    filtered_sequences.append(line)
                else:
                    remaining_sequences.append(line)

    return filtered_sequences, remaining_sequences

# Get the filtered and remaining sequences
filtered_sequences, remaining_sequences = filter_fasta_sequences(fasta_file_path, existing_directories)

# Write the filtered sequences to a new fasta file
with open(filtered_fasta_file_path, 'w') as filtered_file:
    filtered_file.writelines(filtered_sequences)

# Write the remaining sequences back to the original fasta file
with open(updated_fasta_file_path, 'w') as updated_file:
    updated_file.writelines(remaining_sequences)

print("Filtering complete. Filtered sequences written to:", filtered_fasta_file_path)
print("Updated fasta file written to:", updated_fasta_file_path)

Filtering complete. Filtered sequences written to: /home/max/stayahead/out_tmp/alphafold/candidates_100-1.fasta
Updated fasta file written to: /home/max/stayahead/out_tmp/alphafold/candidates_100-2.fasta


In [2]:
# Define the path to the main directory containing the batch directories
main_directory = '/home/max/stayahead/transformers/data/data_glycoprotein/pdb/spike_receptor_binding_pdb'

# Define the path to the new directory where all files will be copied
new_directory = '/home/max/stayahead/transformers/data/data_glycoprotein/pdb/controls_pdb'

# Create the new directory if it doesn't exist
os.makedirs(new_directory, exist_ok=True)

# Iterate over each batch directory
for batch_dir in os.listdir(main_directory):
    batch_dir_path = os.path.join(main_directory, batch_dir)
    
    # Ensure it's a directory
    if os.path.isdir(batch_dir_path):
        # Iterate over each file in the batch directory
        for file_name in os.listdir(batch_dir_path):
            file_path = os.path.join(batch_dir_path, file_name)
            
            # Ensure it's a file
            if os.path.isfile(file_path):
                # Copy the file to the new directory
                shutil.copy(file_path, new_directory)

print("All files have been copied successfully.")

All files have been copied successfully.
